In [1]:
pip install mglearn

Note: you may need to restart the kernel to use updated packages.


In [72]:
%matplotlib inline
from IPython.display import display
import matplotlib_inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mglearn
from cycler import cycler
from sklearn.model_selection import train_test_split
import os

# Libreria per importare CSV da Drive
# from google.colab import drive

matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['image.cmap'] = "viridis"
plt.rcParams['image.interpolation'] = "none"
plt.rcParams['savefig.bbox'] = "tight"
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['legend.numpoints'] = 1
plt.rc('axes', prop_cycle=(
    cycler('color', mglearn.plot_helpers.cm_cycle.colors) +
    cycler('linestyle', ['-', '-', "--", (0, (3, 3)), (0, (1.5, 1.5))])))

np.set_printoptions(precision=3, suppress=True)

pd.set_option("display.max_columns", 16)
pd.set_option('display.precision', 2)

__all__ = ['np', 'mglearn', 'display', 'plt', 'pd']


In [73]:
modelli = ['LogisticRegression', 'KNeighborsClassifier',' DecisionTreeClassifier', 'RandomForestClassifier' ,'SVC', 'GaussianNB', 'LinearDiscriminantAnalysis', 'AdaBoostClassifier' ]
modelli = ['RandomForestClassifier']
def mod(modello):
    if modello == 'LogisticRegression':
        from sklearn.linear_model import LogisticRegression
        model = LogisticRegression()
    
    if modello == 'KNeighborsClassifier':
        from sklearn.neighbors import KNeighborsClassifier
        model = KNeighborsClassifier(n_neighbors=3)

    if modello == 'DecisionTreeClassifier':
        from sklearn.tree import DecisionTreeClassifier
        model = DecisionTreeClassifier()
    if modello == 'RandomForestClassifier':
        from sklearn.ensemble import RandomForestClassifier
        model = RandomForestClassifier()
    if modello == 'SVC':
        from sklearn.svm import SVC
        model = SVC(kernel='linear')
    if modello == 'GaussianNB':
        from sklearn.naive_bayes import GaussianNB
        model = GaussianNB()
    if modello == 'LinearDiscriminantAnalysis':
        from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
        model = LinearDiscriminantAnalysis()
    if modello == 'AdaBoostClassifier':
        from sklearn.ensemble import AdaBoostClassifier
        model = AdaBoostClassifier()
    return model

In [74]:
from sklearn.preprocessing import OneHotEncoder

def ohe(df, list_of_columns):
    # Inizializzare OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    
    # Manteniamo una copia del DataFrame originale
    df_encoded = df.copy()

    # Iterare sulle colonne da codificare
    for col in list_of_columns:
        # Applicare OneHotEncoder alla colonna specifica
        encoded = encoder.fit_transform(df[[col]])

        # Creare un DataFrame con le nuove colonne codificate, usando gli stessi indici del DataFrame originale
        encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([col]), index=df.index)

        # Concatenare le nuove colonne codificate al DataFrame originale
        df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
    df_encoded = df_encoded.drop(list_of_columns, axis=1)

    # Restituire il DataFrame finale, mantenendo le colonne originali
    return df_encoded

In [121]:
dfs = []
for filename in os.listdir():
    if filename.endswith('.csv'):
        print(filename)
        df_ = pd.read_csv(filename).drop(['device_fraud_count', 'month'], axis=1)
        if 'x1' in list(df_.columns):
            df_ = df_.drop(['x1', 'x2'], axis=1)
        df_['fonte'] = filename
        dfs.append(df_)
df = pd.concat(dfs)


Base.csv
Variant I.csv
Variant II.csv
Variant III.csv
Variant IV.csv
Variant V.csv


In [122]:
lista_fonti = ['Base.csv', 'Variant I.csv', 'Variant II.csv','Variant III.csv','Variant IV.csv','Variant V.csv']
diz_df = dict()
for l in lista_fonti:
    diz_df[l] = df[(df['fonte'] == l)].drop(['fonte'], axis=1).drop_duplicates()

In [123]:
diz_df.keys()

dict_keys(['Base.csv', 'Variant I.csv', 'Variant II.csv', 'Variant III.csv', 'Variant IV.csv', 'Variant V.csv'])

In [124]:
df = diz_df['Base.csv']
df_frodi = df[(df['fraud_bool'] == 1)]
n = df[(df['fraud_bool'] == 1)].shape[0]
print(n)
df_not_frodi = df[(df['fraud_bool'] != 1)]#.head(n*50)

df = pd.concat([df_frodi, df_not_frodi])


11029


In [125]:
def train(df):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier()
    from sklearn.model_selection import train_test_split
    df = ohe(df,['employment_status','payment_type','housing_status', 'source', 'device_os'])
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['fraud_bool'], axis=1), df['fraud_bool'], random_state=0)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuratezza: {accuracy}')

    # Calcolare la precisione
    precision = precision_score(y_test, y_pred)
    print(f'Precisione: {precision}')

    # Calcolare il recall (sensibilità)
    recall = recall_score(y_test, y_pred)
    print(f'Recall: {recall}')

    # Matrice di confusione
    cm = confusion_matrix(y_test, y_pred)
    print(f'Matrice di confusione:\n{cm}')
    return model

In [126]:
def test(df, model):
    
    df = ohe(df,['employment_status','payment_type','housing_status', 'source', 'device_os'])
    y_pred = model.predict(df.drop(['fraud_bool'], axis=1))
    y_test = df['fraud_bool']

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuratezza: {accuracy}')

    # Calcolare la precisione
    precision = precision_score(y_test, y_pred)
    print(f'Precisione: {precision}')

    # Calcolare il recall (sensibilità)
    recall = recall_score(y_test, y_pred)
    print(f'Recall: {recall}')

    # Matrice di confusione
    cm = confusion_matrix(y_test, y_pred)
    print(f'Matrice di confusione:\n{cm}')

    print('\n__________\n')

In [127]:
model = train(df)

Accuratezza: 0.988976
Precisione: 0.8333333333333334
Recall: 0.0018115942028985507
Matrice di confusione:
[[247239      1]
 [  2755      5]]


In [128]:
for fonte in lista_fonti:
    print('\n\n', fonte)
    test(diz_df[fonte], model)



 Base.csv
Accuratezza: 0.997227
Precisione: 0.9998789053039477
Recall: 0.7486626167376915
Matrice di confusione:
[[988970      1]
 [  2772   8257]]

__________



 Variant I.csv
Accuratezza: 0.990196
Precisione: 0.9983726606997559
Recall: 0.11125215341372745
Matrice di confusione:
[[988969      2]
 [  9802   1227]]

__________



 Variant II.csv
Accuratezza: 0.990184
Precisione: 0.9902991107518189
Recall: 0.11107081331036359
Matrice di confusione:
[[988959     12]
 [  9804   1225]]

__________



 Variant III.csv
Accuratezza: 0.990075
Precisione: 0.9893711248892826
Recall: 0.10126926563916591
Matrice di confusione:
[[988958     12]
 [  9913   1117]]

__________



 Variant IV.csv
Accuratezza: 0.990086
Precisione: 0.9894736842105263
Recall: 0.10226654578422484
Matrice di confusione:
[[988958     12]
 [  9902   1128]]

__________



 Variant V.csv
Accuratezza: 0.990072
Precisione: 0.9867491166077739
Recall: 0.10126926563916591
Matrice di confusione:
[[988955     15]
 [  9913   1117]]

